# Cleanup and Exploration of 2016 AQ Data

In [1]:
import pandas as pd
import json as json

In [2]:
df = pd.read_csv('2016 AQ.csv')

C:\Users\mhenr\AppData\Local\Temp\ipykernel_42496\1323086204.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2016 AQ.csv')


In [3]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,AirNow,NaN,NaN
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,AirNow,NaN,NaN
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,AirNow,NaN,NaN


The entry is not valid json.  The field will need processing code.

In [3]:
def validate_json_fields(field: str) -> bool:
    """This function validates a JSON entry is valid.  Returns True if so, False if otherwise."""
    try:
        json.loads(field)
        return True
    except ValueError:
        return False

In [4]:
def validate_json_column(series: pd.Series):
    temp_df = pd.DataFrame()
    temp_df['values'] = series
    temp_df['validity'] = series.apply(validate_json_fields)
    return temp_df

## Validation of composite fields as json

In [5]:
output = validate_json_column(df['date'])

In [6]:
output.head()

,values,validity
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",False
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",False
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",False
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",False
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",False


In [7]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [9]:
output = validate_json_column(df['attribution'])
output.head()

,values,validity
0,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
1,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
2,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
3,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
4,"[{name=US EPA AirNow, url=http://www.airnow.go...",False


In [10]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [11]:
output = validate_json_column(df['averagingperiod'])
output.head()

,values,validity
0,"{unit=hours, value=1.0}",False
1,"{unit=hours, value=1.0}",False
2,"{unit=hours, value=1.0}",False
3,"{unit=hours, value=1.0}",False
4,"{unit=hours, value=1.0}",False


In [12]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [12]:
output = validate_json_column(df['coordinates'])
output.head()

,values,validity
0,"{latitude=35.5036, longitude=-119.2717}",False
1,"{latitude=34.1992, longitude=-118.5331}",False
2,"{latitude=34.1992, longitude=-118.5331}",False
3,"{latitude=33.5531, longitude=-86.815}",False
4,"{latitude=33.3311, longitude=-87.0036}",False


In [13]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


All of the composite fields are in a "JSON-like format"  need to convert to JSON.

## Reprocessing the Multivalue Fields into Valid JSON

In [14]:
def correct_field_to_json(arg: str) -> str:
    string = arg.lstrip("{{")
    string = string.rstrip("]}")
    strings = string.split(",")
    new_strings = []
    for entry in strings:
        index = entry.find('=')
        entry = '"' + entry[0:index] + '"' + ':' + '"'+ entry[index+1:len(entry)]+'"'
        new_strings.append(entry)
    output_string = (',').join(entry for entry in new_strings)
    output_string = '{' + output_string + '}'
    return output_string

In [15]:
def correct_json_column(series: pd.Series, column_name: str) -> pd.DataFrame:
    temp_df = pd.DataFrame()
    temp_df[column_name] = series.apply(correct_field_to_json)
    return temp_df

In [16]:
test = correct_json_column(df['date'], 'test date conversion')
results = validate_json_column(test['test date conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [17]:
df['cleaneddate'] = test

In [18]:
test = correct_json_column(df['attribution'], 'test attribute conversion')
results = validate_json_column(test['test attribute conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [19]:
results.head()

,values,validity
0,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
1,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
2,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
3,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
4,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True


In [20]:
df['cleanedattribution'] = test

In [21]:
test = correct_json_column(df['averagingperiod'], 'test averaging period conversion')
results = validate_json_column(test['test averaging period conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [22]:
df['cleanedaveragingperiod'] = test

In [23]:
test = correct_json_column(df['coordinates'], 'test coordinate conversion')
results = validate_json_column(test['test coordinate conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [24]:
df['cleanedcoordinates'] = test

In [25]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}"
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}"
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}"
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}"
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}"


In [26]:
df_trans = df
import ujson as ujson

In [27]:
#df_trans = pd.concat([df_trans, df_trans.cleanedcoordinates.apply(json.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df_trans['cleanedcoordinates'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df_trans = pd.concat([df_trans, expanded_datascope], axis =1)

In [28]:
df_trans.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036


In [29]:
#df_trans = pd.concat([df_trans, df_trans.cleaneddate.apply(ujson.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df_trans['cleaneddate'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df_trans = pd.concat([df_trans, expanded_datascope], axis =1)

In [30]:
df_trans.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,...,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,...,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,...,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00


In [31]:
#df_trans = pd.concat([df_trans, df_trans.cleanedattribution.apply(json.loads).apply(pd.Series)], axis = 1) This line is EXTREMELY SLOW TRYING TO OPTIMIZE.
expanded_datascope = df_trans['cleanedattribution'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df_trans = pd.concat([df_trans, expanded_datascope], axis =1)

In [32]:
df_trans.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local,[{name,url,{name
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,...,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,California Air Resources Board
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,South Coast AQMD
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,South Coast AQMD
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,...,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,Jefferson County Department of Health
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,...,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,Jefferson County Department of Health


In [33]:
df_trans.drop(' {name', axis=1, inplace=True)

In [34]:
df_trans.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local,[{name,url
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,...,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,...,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,...,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null


In [35]:
#df_trans = pd.concat([df_trans, df_trans.cleanedaveragingperiod.apply(json.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df_trans['cleanedaveragingperiod'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df_trans = pd.concat([df_trans, expanded_datascope], axis =1)

In [36]:
df_trans.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local,[{name,url,unit,value
0,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,"{utc=2016-03-19T16:00:00.000Z, local=2016-03-1...",o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [37]:
df_trans.drop('date', axis =1, inplace=True)

In [38]:
df_trans.head()

,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,...,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local,[{name,url,unit,value
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=35.5036, longitude=-119.2717}",US,AirNow,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""35.5036"","" longitude"":""-119.2717""}",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.1992, longitude=-118.5331}",US,AirNow,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.1992"","" longitude"":""-118.5331""}",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.5531, longitude=-86.815}",US,AirNow,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.5531"","" longitude"":""-86.815""}",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.3311, longitude=-87.0036}",US,AirNow,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.3311"","" longitude"":""-87.0036""}",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [39]:
df_trans.drop(['attribution','averagingperiod','coordinates','cleanedaveragingperiod','cleanedcoordinates'], axis = 1, inplace = True)

In [40]:
df_trans.head()

,parameter,location,value,unit,city,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,latitude,longitude,utc,local,[{name,url,unit,value
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,"{""utc"":""2016-03-19T16:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [41]:
df_trans.drop(['cleaneddate','cleanedattribution'], axis = 1, inplace = True)

In [42]:
df_trans.head()

,parameter,location,value,unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,[{name,url,unit,value
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,US,AirNow,NaN,NaN,35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [43]:
#df_trans.rename(columns={df_trans.columns[15]: 'averaging time unit'}, inplace=True)
df_trans.columns.values[3]='concentration unit'

In [44]:
df_trans.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,[{name,url,unit,value
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,US,AirNow,NaN,NaN,35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [45]:
#df_trans.rename(columns={df_trans.columns[15]: 'averaging time unit'}, inplace=True)
df_trans.columns.values[15] = 'averaging time unit'

In [46]:
#df_trans.rename(columns={df_trans.columns[13]: 'source'}, inplace=True)
df_trans.columns.values[13] = 'source'

In [47]:
df_trans.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,averaging time unit,value
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,US,AirNow,NaN,NaN,35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [48]:
#df_trans.rename(columns={df_trans.columns[16]: 'averaging time'},inplace=True)
df_trans.columns.values[16] = 'averaging time'

In [49]:
df_trans.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,averaging time unit,averaging time
0,o3,Shafter - Walker Str,0.023,ppm,Bakersfield,US,AirNow,NaN,NaN,35.5036,-119.2717,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
1,pm25,Reseda,48.000,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
2,o3,Reseda,0.030,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,NaN,NaN,34.1992,-118.5331,2016-03-19T16:00:00.000Z,2016-03-19T08:00:00-08:00,US EPA AirNow,null,hours,1.0
3,o3,NO. BHAM,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.5531,-86.815,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0
4,o3,MCADORY,0.030,ppm,Birmingham-Hoover,US,AirNow,NaN,NaN,33.3311,-87.0036,2016-03-19T16:00:00.000Z,2016-03-19T10:00:00-06:00,US EPA AirNow,null,hours,1.0


In [50]:
df_trans.to_csv('2016 AQ Clean.csv', index=False)